<h1>SI 370 Homework 7</h1>
<h2>Class Cheat Sheet: 6 Steps to Data Science</h2>
<p>
    Haley Johnson<br>
    Data source: <a href = "https://www.kaggle.com/datasets/thedevastator/scraping-apartments-off-of-craigslist-in-san-fra">Housing Prices in San Francisco (Craigslist)</a> hosted on Kaggle 
    

</p>

In [123]:
import pandas as pd
import numpy as np 
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from sklearn import manifold

In [81]:
df = pd.read_csv('data/raw_sf_scrape.csv')

In [82]:
df.head()

,date,title,link,price,brs,sqft,hood,bath,amenities
0,Oct 1,"3D Virtual Tour - 2 BR, 2 BA Condo 966 Sq. Ft....",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,3850,2.0,966.0,Mission Bay,2Ba,"['condo', 'w/d in unit', 'attached garage']"
1,Oct 1,Beautiful house for rent,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,900,1.0,NaN,portola district,0Ba,['house']
2,Oct 1,4 BEDROOM APARTMENT IN THE HAIGHT,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,4995,4.0,NaN,haight ashbury,2Ba,"['apartment', 'w/d in unit', 'street parking']"
3,Oct 1,4 BEDROOM APARTMENT IN THE HAIGHT,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,4995,4.0,NaN,haight ashbury,2Ba,"['apartment', 'w/d in unit', 'street parking']"
4,Oct 1,ENJOY GOLDEN GATE PARK EVERYDAY,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2800,2.0,700.0,USF / panhandle,1Ba,"['apartment', 'laundry in bldg', 'no smoking',..."


In [83]:
df.columns = ['date', 'title', 'link', 'price', 'bedrooms', 'square_feet', 'neighborhood', 'bathrooms', 'amenities']

<h3>1: Exploratory Data Analysis</h3>

<p>Collect basic information about the dataset</p>

In [33]:
n_rows, n_columns = df.shape
print(f"There are {n_rows} rows and {n_columns} columns in the dataset")

There are 3093 rows and 9 columns in the dataset


In [34]:
columns = df.columns

for column in columns:
    missing_values = pd.isnull(df[column]).sum()
    print(f"There are {missing_values} in {column}")

There are 0 in date
There are 0 in title
There are 0 in link
There are 0 in price
There are 617 in brs
There are 1632 in sqft
There are 31 in hood
There are 51 in bath
There are 0 in amenities


In [35]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,3093.0,3318.156159,1710.975464,1.0,2395.0,2995.0,3933.0,29000.0
brs,2476.0,1.836430,0.948098,1.0,1.0,2.0,2.0,7.0
sqft,1461.0,1202.036277,1316.612823,110.0,650.0,902.0,1240.0,11020.0


By default, <code>.describe()</code> only includes numeric variables. We can use keyword arguments to change what it outputs.

In [36]:
df.describe(include=np.number)

,price,brs,sqft
count,3093.000000,2476.000000,1461.000000
mean,3318.156159,1.836430,1202.036277
std,1710.975464,0.948098,1316.612823
min,1.000000,1.000000,110.000000
25%,2395.000000,1.000000,650.000000
50%,2995.000000,2.000000,902.000000
75%,3933.000000,2.000000,1240.000000
max,29000.000000,7.000000,11020.000000


In [37]:
df.describe(exclude=np.number)

,date,title,link,hood,bath,amenities
count,3093,3093,3093,3062,3042,3093
unique,2,2658,2879,113,12,923
top,Oct 1,1 bedroom/1 bath in Polk,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,SOMA / south beach,1Ba,['apartment']
freq,2857,17,2,342,2140,115


In [39]:
df.date.describe()

count       3093
unique         2
top       Oct  1
freq        2857
Name: date, dtype: object

In [41]:
df.hood.value_counts()

SOMA / south beach                342
mission district                  251
pacific heights                   190
nob hill                          177
lower nob hill                    163
                                 ... 
San Franciso Richmond District      1
san francisco                       1
Tenderloin                          1
SAN FRANCISCO                       1
Cow Hollow                          1
Name: hood, Length: 113, dtype: int64

In [57]:
df.groupby('bedrooms')['price'].agg(['max', 'mean'])

,max,mean
bedrooms,,
1.0,7200,2781.439312
2.0,26500,3829.589120
3.0,29000,4727.661111
4.0,16995,5564.831858
5.0,17000,6955.862069
6.0,10000,7424.750000
7.0,9995,9995.000000


In [58]:
neighborhood_1 = df[df.neighborhood == 'mission district']
neighborhood_2 = df[df.neighborhood == 'haight ashbury']

along_rows = pd.concat([neighborhood_1, neighborhood_2], axis = 0)

In [65]:
neighborhood_1.merge(neighborhood_2, on = 'date')

,date,title_x,link_x,price_x,bedrooms_x,square_feet_x,neighborhood_x,bathroom_x,amenities_x,title_y,link_y,price_y,bedrooms_y,square_feet_y,neighborhood_y,bathroom_y,amenities_y
0,Oct 1,"Renovated 2bed/1bath Apartment, Laundry in Uni...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2850,2.0,NaN,mission district,1Ba,['application fee details: $30 application fee...,4 BEDROOM APARTMENT IN THE HAIGHT,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,4995,4.0,NaN,haight ashbury,2Ba,"['apartment', 'w/d in unit', 'street parking']"
1,Oct 1,"Renovated 2bed/1bath Apartment, Laundry in Uni...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2850,2.0,NaN,mission district,1Ba,['application fee details: $30 application fee...,4 BEDROOM APARTMENT IN THE HAIGHT,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,4995,4.0,NaN,haight ashbury,2Ba,"['apartment', 'w/d in unit', 'street parking']"
2,Oct 1,"Renovated 2bed/1bath Apartment, Laundry in Uni...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2850,2.0,NaN,mission district,1Ba,['application fee details: $30 application fee...,($3000)Look&Lease Special! Newly Renovated 4BD...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,5595,4.0,NaN,haight ashbury,2Ba,['application fee details: $30 application fee...
3,Oct 1,"Renovated 2bed/1bath Apartment, Laundry in Uni...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2850,2.0,NaN,mission district,1Ba,['application fee details: $30 application fee...,HUGE 5 Bedroom w/ Condo Level Finishes near Ha...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,6895,5.0,NaN,haight ashbury,2Ba,"['application fee details: $30 per adult', 'ca..."
4,Oct 1,"Renovated 2bed/1bath Apartment, Laundry in Uni...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2850,2.0,NaN,mission district,1Ba,['application fee details: $30 application fee...,Beautiful 4BR in the Haight,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,4295,4.0,NaN,haight ashbury,1Ba,"['application fee details: $30 per adult', 'ca..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8000,Sep 30,Bright Flat with Large Rooms/2baths/Newer Kitc...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,5695,4.0,1965.0,mission district,2Ba,"['cats are OK - purrr', 'flat', 'laundry in bl...","SPACIOUS 1BR w/ Brand New Hardwood Floors, Pet...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2900,1.0,600.0,haight ashbury,1Ba,"['cats are OK - purrr', 'dogs are OK - wooof',..."
8001,Sep 30,Bright Flat with Large Rooms/2baths/Newer Kitc...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,5695,4.0,1965.0,mission district,2Ba,"['cats are OK - purrr', 'flat', 'laundry in bl...",2-story Victorian with views,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,6875,4.0,3000.0,haight ashbury,3Ba,"['flat', 'w/d in unit', 'no smoking', 'carport']"
8002,Sep 30,Bright Flat with Large Rooms/2baths/Newer Kitc...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,5695,4.0,1965.0,mission district,2Ba,"['cats are OK - purrr', 'flat', 'laundry in bl...",2-story Victorian with views,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,6875,4.0,3000.0,haight ashbury,3Ba,"['flat', 'w/d in unit', 'no smoking', 'carport']"
8003,Sep 30,Bright Flat with Large Rooms/2baths/Newer Kitc...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,5695,4.0,1965.0,mission district,2Ba,"['cats are OK - purrr', 'flat', 'laundry in bl...",NICE ONE BED ONE BATH WITH PARKING (haight Ash...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2395,1.0,700.0,haight ashbury,1Ba,"['apartment', 'laundry in bldg', 'no smoking',..."


In [70]:
crosstab = pd.crosstab(index = df.bedrooms, columns = df.bathrooms, values = df.price, aggfunc = np.median)

In [ ]:
x_dim, y_dim = 18, 24

sns.set(rc={'figure.figsize':(x_dim, y_dim)})

In [ ]:
n_rows, n_cols = 1, 2

f, axes = plt.subplots(n_rows, n_cols)

In [104]:
from scipy.stats import chi2_contingency
import scipy
import statsmodels

In [85]:
chi2, p_val, degrees_freedom, exp_val = scipy.stats.chi2_contingency(pd.crosstab(df.bedrooms, df.bathrooms))

In [101]:
df.bathrooms.value_counts()

df.bathrooms = pd.Series(np.where(df.bathrooms == 'splitBa', '0.5Ba', df.bathrooms))
df.bathrooms = pd.Series(np.where(df.bathrooms == 'sharedBa', '0.5Ba', df.bathrooms))
df.bathrooms = df.bathrooms.str.replace('Ba', '', regex = True).astype(float)

In [113]:
lm = statsmodels.formula.api.ols('price ~ bathrooms + bedrooms + square_feet + C(neighborhood)', data = df).fit()

<h3>Dimension Reducation</h3>

In [131]:
df['bedrooms'] = df['bedrooms'].fillna(df['bedrooms'].median())
df['square_feet'] = df['square_feet'].fillna(df['square_feet'].median())
df['bathrooms'] = df['bathrooms'].fillna(df['bathrooms'].median())

In [132]:
X = df[['price', 'bedrooms', 'square_feet', 'bathrooms']]
mds = manifold.MDS(n_components = 2, metric = False, eps = 1e-9, random_state = 42, dissimilarity = 'euclidean', n_jobs = 1)
mds = mds.fit_transform(X)

In [ ]:
npos_labelled = pd.concat([pd.DataFrame({'who':demo_y}),pd.DataFrame(npos)],axis=1)
npos_labelled.columns = ['who','mds1','mds2']